In [1]:
import lightning as L
import torchvision
import torch

In [2]:
from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.IMAGENET1K_V1
preprocess_tranform = weights.transforms()
preprocess_tranform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [3]:
pytorch_model = torch.hub.load("pytorch/vision", "resnet18", weights="IMAGENET1K_V1")

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_main


In [4]:
from data_loader.CIFAR10 import CIFAR10

L.pytorch.seed_everything(42)
dm = CIFAR10(batch_size=64, data_path="./data/cifar10/")
dm.prepare_data()
dm.setup()

Seed set to 42


Files already downloaded and verified


In [5]:
pytorch_model.fc = torch.nn.Linear(pytorch_model.fc.in_features, dm.num_classes)

In [6]:
dm = CIFAR10(
    height_width=(224, 224),
    batch_size=64,
    data_path="./data/cifar10/",
    num_workers=4,
    train_transform=preprocess_tranform,
    test_transform=preprocess_tranform,
)

In [ ]:
lightning_model = LightningModel